## Method2Test Breakdown

In [2]:
import os
import json
import pandas as pd
from glob import glob
import numpy

# Define the base directories
base_dirs = ['train', 'eval', 'test']

for i in range(len(base_dirs)):
    base_dirs[i] = 'data/methods2test_data/' + base_dirs[i]
# List to collect all data (rows)
data = []

total_exmaples = 0

is_broken = False
base_dir = 'data/methods2test_data/train'
# Iterate over all subdirectories (examples) in the base directory
for example_dir in os.listdir(base_dir):
    example_path = os.path.join(base_dir, example_dir)
    
    # Ensure we're dealing with directories (i.e., each example is its own directory)
    if os.path.isdir(example_path):
        # Use glob to get all JSON files in the example directory
        json_files = glob(os.path.join(example_path, '*.json'))
        
        # Iterate over the JSON files
        for json_file in json_files:
            # Read the JSON data
            with open(json_file, 'r') as f:
                json_data = json.load(f)
            
            # Add the JSON data to the data list (can modify if additional info is needed)
            data.append(json_data)
            total_exmaples+=1
            if(total_exmaples>=100):
                print("breaking")
                is_broken = True
                break
    if(is_broken):
        break
# Convert the collected data into a pandas DataFrame
df = pd.DataFrame(data)

# Optional: Inspect the first few rows
#print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'data/methods2test_data/train'

In [1]:
df.iloc[1].iloc[0]

NameError: name 'df' is not defined

In [ ]:
df.iloc[1].iloc[1]

In [ ]:
df.iloc[1].iloc[2]

In [ ]:
df.iloc[1].iloc[3]

In [ ]:
df.iloc[1].iloc[4]

In [10]:
from collections import Counter
total_rows = 0
counter = Counter()
for row in df['repository']:
    total_rows += 1
    if isinstance(row, dict):
        value = row.get('language')
        if value is not None:
            counter[value] += 1
    elif isinstance(row, list):
        for item in row:
            value = item.get('language')
            if value is not None:
                counter[value] += 1
counter

Counter({'Java': 15})

In [11]:
total_rows

15

### Inspecting Source Code vs Test Code and how they relate

In [ ]:
df['test_case']

### Pull all Code Bodies as Parameters for the Model


In [2]:
focal_methods = df['focal_method']
code_bodies = pd.DataFrame({'code': focal_methods.apply(lambda x: x['body'])})

In [ ]:
code_bodies.info()

In [ ]:
test_cases_classes = df['test_case']

In [3]:
test_cases = df["test_case"]
test_code_bodies = pd.DataFrame({'tests': test_cases.apply(lambda x: x['body'])})

In [ ]:
test_code_bodies.info()

In [ ]:
code_test_df = pd.concat([code_bodies, test_code_bodies], axis=1)
code_test_df.info()

## Add Prompts Before Each Code Body

In [5]:
code_test_df['prompted_code'] = "Here is a method implementation in Java:\n\n" + code_test_df['code'] + "\n\nWrite a full test class with test cases to validate the method defined above."

## Test Model Responses
* StarCoder
* CodeLlama
* DeepSeek-Coder

### StarCoder

In [47]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "bigcode/starcoder2-7b"
device = "cuda:0"

starcoder_tokenizer = AutoTokenizer.from_pretrained(model_name)
starcoder_tokenizer.pad_token = starcoder_tokenizer.eos_token

starcoder_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Loading checkpoint shards: 100%|██████████| 3/3 [00:28<00:00,  9.47s/it]


#### Fibonacci Example

In [48]:
prompt = "Create a fibonacci sequence method in Java and write JUnit test cases to verify the code runs and computes correctly."#code_test_df["prompted_code"].iloc[7]

inputs = starcoder_tokenizer(prompt,
                             padding=True,
                             truncation=True,
                             return_tensors="pt")

inputs = {key: value.to(device) for key, value in inputs.items()}

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
from transformers.generation.utils import GenerationConfig

config = starcoder_model.config

generation_config = GenerationConfig.from_model_config(config)
print(generation_config)

In [ ]:
vars(generation_config)

In [21]:
outputs = starcoder_model.generate(
    inputs['input_ids'], 
    max_length=1000,
    num_return_sequences=1,
    temperature=0.7,
    eos_token_id=starcoder_tokenizer.eos_token_id
)

generated_text = starcoder_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

/home/nzappia/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Here is a method implementation in Java:

public static void checkIdentifierLetterAndDigitRequirements(String identifier) {
        // An identifier is an unlimited-length sequence of letters and digits, the first of
        // which must be a letter.  A letter is any character for which the method
        // Character.isJavaLetter returns true.  This includes '_' and '$'.  A letter or digit
        // is any character for which the method Character.isJavaLetterOrDigit returns true.
        char startChar = identifier.charAt(0);
        if (!(Character.isJavaIdentifierStart(startChar))) {
            throw new IllegalArgumentException("Identifier does not begin with a valid JMS identifier start character: '" + identifier + "' ");
        }

        // JMS part character
        int length = identifier.length();
        for (int i = 1; i < length; i++) {
            char ch = identifier.charAt(i);
            if (!(Character.isJavaIdentifierPart(ch))) {
                throw new Illegal

In [3]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)

2.5.1
True
12.4


#### Example from Dataset

In [48]:
prompt = code_test_df["prompted_code"].iloc[11]
target = code_test_df['tests'].iloc[11]

print(prompt)

inputs = starcoder_tokenizer(prompt,
                             padding=True,
                             truncation=True,
                             return_tensors="pt")

inputs = {key: value.to(device) for key, value in inputs.items()}

outputs = starcoder_model.generate(
    inputs['input_ids'], 
    max_length=2000,
    #num_return_sequences=1,
    temperature = 0.7,
    eos_token_id=starcoder_tokenizer.eos_token_id
)

generated_text = starcoder_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Here is a method implementation in Java:

public static void checkIdentifierLetterAndDigitRequirements(String identifier) {
        // An identifier is an unlimited-length sequence of letters and digits, the first of
        // which must be a letter.  A letter is any character for which the method
        // Character.isJavaLetter returns true.  This includes '_' and '$'.  A letter or digit
        // is any character for which the method Character.isJavaLetterOrDigit returns true.
        char startChar = identifier.charAt(0);
        if (!(Character.isJavaIdentifierStart(startChar))) {
            throw new IllegalArgumentException("Identifier does not begin with a valid JMS identifier start character: '" + identifier + "' ");
        }

        // JMS part character
        int length = identifier.length();
        for (int i = 1; i < length; i++) {
            char ch = identifier.charAt(i);
            if (!(Character.isJavaIdentifierPart(ch))) {
                throw new Illegal

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Here is a method implementation in Java:

public static void checkIdentifierLetterAndDigitRequirements(String identifier) {
        // An identifier is an unlimited-length sequence of letters and digits, the first of
        // which must be a letter.  A letter is any character for which the method
        // Character.isJavaLetter returns true.  This includes '_' and '$'.  A letter or digit
        // is any character for which the method Character.isJavaLetterOrDigit returns true.
        char startChar = identifier.charAt(0);
        if (!(Character.isJavaIdentifierStart(startChar))) {
            throw new IllegalArgumentException("Identifier does not begin with a valid JMS identifier start character: '" + identifier + "' ");
        }

        // JMS part character
        int length = identifier.length();
        for (int i = 1; i < length; i++) {
            char ch = identifier.charAt(i);
            if (!(Character.isJavaIdentifierPart(ch))) {
                throw new Illegal

### BLEU Score

In [13]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(predicted: str, target: str) -> float:
    """
    Calculate the BLEU score between a predicted string and a target string.

    Args:
        predicted (str): The predicted sentence.
        target (str): The target (reference) sentence.

    Returns:
        float: The BLEU score between 0 and 1.
    """
    # Tokenize the strings
    predicted_tokens = predicted.split()
    target_tokens = target.split()

    # BLEU expects references as a list of lists
    reference = [target_tokens]
    
    # Calculate BLEU score using smoothing function
    smoothing_function = SmoothingFunction().method1
    bleu_score = sentence_bleu(reference, predicted_tokens, smoothing_function=smoothing_function)

    return bleu_score

bleu = calculate_bleu(generated_text, target)
print(f"BLEU score: {bleu:.4f}")
print(f"Output: ", generated_text)
print("Target: ", target)


ModuleNotFoundError: No module named 'nltk'

### CSR (Compliation Success Rate) for 1 example

In [10]:
import subprocess
import os

def java_pipeline(java_code: str, class_name: str):
    """
    Takes a Java code string, writes it to a .java file, compiles it, runs it, and tracks the results.

    Args:
        java_code (str): The Java code as a string.
        class_name (str): The name of the Java class (should match the public class name).

    Returns:
        dict: Compilation and execution results including success status and error messages (if any).
    """
    file_name = f"{class_name}.java"
    results = {"compilation_success": False, "execution_success": False, "compilation_error": None, "execution_error": None, "output": None}

    try:
        # Write the Java code to a file
        with open(file_name, "w") as file:
            file.write(java_code)
        
        # Compile the Java file
        compile_process = subprocess.run(
            ["javac", file_name],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        if compile_process.returncode != 0:
            results["compilation_error"] = compile_process.stderr
            return results
        
        results["compilation_success"] = True

        # Run the compiled Java class
        run_process = subprocess.run(
            ["java", class_name],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        if run_process.returncode != 0:
            results["execution_error"] = run_process.stderr
        else:
            results["execution_success"] = True
            results["output"] = run_process.stdout

    except Exception as e:
        results["compilation_error"] = str(e)

    finally:
        # Cleanup: Remove the generated files
        if os.path.exists(file_name):
            os.remove(file_name)
        class_file = f"{class_name}.class"
        if os.path.exists(class_file):
            os.remove(class_file)

    return results

# Example Usage
java_code = """
public class HelloWorld {
    public static void main(String[] args) {
        System.out.println("Hello, world!");
    }
}
"""

class_name = "HelloWorld"
result = java_pipeline(java_code, class_name)

# Display results
print("Compilation Success:", result["compilation_success"])
if result["compilation_error"]:
    print("Compilation Error:", result["compilation_error"])

print("Execution Success:", result["execution_success"])
if result["execution_error"]:
    print("Execution Error:", result["execution_error"])

if result["output"]:
    print("Output:", result["output"])


Compilation Success: True
Execution Success: True
Output: Hello, world!



In [17]:
print(generated_text)

Here is a method implementation in Java:

public static void checkIdentifierLetterAndDigitRequirements(String identifier) {
        // An identifier is an unlimited-length sequence of letters and digits, the first of
        // which must be a letter.  A letter is any character for which the method
        // Character.isJavaLetter returns true.  This includes '_' and '$'.  A letter or digit
        // is any character for which the method Character.isJavaLetterOrDigit returns true.
        char startChar = identifier.charAt(0);
        if (!(Character.isJavaIdentifierStart(startChar))) {
            throw new IllegalArgumentException("Identifier does not begin with a valid JMS identifier start character: '" + identifier + "' ");
        }

        // JMS part character
        int length = identifier.length();
        for (int i = 1; i < length; i++) {
            char ch = identifier.charAt(i);
            if (!(Character.isJavaIdentifierPart(ch))) {
                throw new Illegal

In [15]:
result = java_pipeline(generated_text, "Test")
print("Compilation Success:", result["compilation_success"])
if result["compilation_error"]:
    print("Compilation Error:", result["compilation_error"])

print("Execution Success:", result["execution_success"])
if result["execution_error"]:
    print("Execution Error:", result["execution_error"])

if result["output"]:
    print("Output:", result["output"])

Compilation Success: False
Compilation Error: Test.java:1: error: class, interface, or enum expected
Here is a method implementation in Java:
^
Test.java:3: error: class, interface, or enum expected
public static void checkIdentifierLetterAndDigitRequirements(String identifier) {
              ^
Test.java:9: error: class, interface, or enum expected
        if (!(Character.isJavaIdentifierStart(startChar))) {
        ^
Test.java:11: error: class, interface, or enum expected
        }
        ^
Test.java:15: error: class, interface, or enum expected
        for (int i = 1; i < length; i++) {
        ^
Test.java:15: error: class, interface, or enum expected
        for (int i = 1; i < length; i++) {
                        ^
Test.java:15: error: class, interface, or enum expected
        for (int i = 1; i < length; i++) {
                                    ^
Test.java:17: error: class, interface, or enum expected
            if (!(Character.isJavaIdentifierPart(ch))) {
            ^
Tes

### Remove Model from Memory

In [46]:
del starcoder_model
del starcoder_tokenizer

import torch
torch.cuda.empty_cache()

NameError: name 'starcoder_model' is not defined

## CodeLlama

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "codellama/CodeLlama-7b-Instruct-hf"
device = "cuda:0"

codellama_tokenizer = AutoTokenizer.from_pretrained(model_name)
codellama_tokenizer.pad_token = codellama_tokenizer.eos_token

codellama_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto").to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.16it/s]


#### Fibonacci Example

In [53]:
prompt = "Create a fibonacci sequence method in Java and write JUnit test cases to verify the code runs and computes correctly."#code_test_df["prompted_code"].iloc[7]

inputs = codellama_tokenizer(prompt,
                             padding=True,
                             truncation=True,
                             return_tensors="pt")

inputs = {key: value.to(device) for key, value in inputs.items()}

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [54]:
outputs = codellama_model.generate(
    inputs['input_ids'], 
    max_length=1000,
    num_return_sequences=1,
    temperature=0.7,
    eos_token_id=codellama_tokenizer.eos_token_id
)

generated_text = codellama_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

/home/nzappia/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Create a fibonacci sequence method in Java and write JUnit test cases to verify the code runs and computes correctly.

### Fibonacci Sequence

The Fibonacci sequence is a series of numbers where each number is the sum of the two preceding numbers. The sequence starts with 0 and 1, and each subsequent number is the sum of the two preceding numbers.

```
0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20365011074, 32951280099, 53316291173, 86267571272, 139583862445, 225851433717, 365435296162, 591286729879, 956722026049, 1548008755920, 2504730781961, 4052739537881, 6597203966401, 10600418121409, 16777216000000
```

### JUnit Test Cases

```
@Test
public void testF

#### Example from Dataset

In [29]:
prompt = code_test_df["prompted_code"].iloc[11]

inputs = codellama_tokenizer(prompt,
                             padding=True,
                             truncation=True,
                             return_tensors="pt")

inputs = {key: value.to(device) for key, value in inputs.items()}

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [32]:
outputs = codellama_model.generate(
    inputs['input_ids'], 
    max_length=2000,
    num_return_sequences=1,
    temperature=0.7,
    eos_token_id=codellama_tokenizer.eos_token_id
)

generated_text = codellama_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text[len(prompt):].strip())

/media/mujtaba/DATA/nick/miniconda3/envs/UnitTestGeneration/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


package com.java.test;

import org.junit.Test;

public class IdentifierTest {

    @Test
    public void testValidIdentifier() {
        IdentifierTest.checkIdentifierLetterAndDigitRequirements("abc123");
    }

    @Test
    public void testValidIdentifierWithUnderscore() {
        IdentifierTest.checkIdentifierLetterAndDigitRequirements("abc_123");
    }

    @Test
    public void testValidIdentifierWithDollar() {
        IdentifierTest.checkIdentifierLetterAndDigitRequirements("abc$123");
    }

    @Test
    public void testInvalidIdentifierWithDigitFirst() {
        try {
            IdentifierTest.checkIdentifierLetterAndDigitRequirements("1abc");
        } catch (IllegalArgumentException e) {
            return;
        }
        throw new AssertionError("Expected IllegalArgumentException");
    }

    @Test
    public void testInvalidIdentifierWithDigit() {
        try {
            IdentifierTest.checkIdentifierLetterAndDigitRequirements("abc1");
        } catch (IllegalArgume

In [20]:
codellama_model.config.torch_dtype

torch.bfloat16

In [23]:
import torch
def supports_flash_attention(device_id):
    """Check if a GPU supports FlashAttention."""
    major, minor = torch.cuda.get_device_capability(device_id)
    
    # Check if the GPU architecture is Ampere (SM 8.x) or newer (SM 9.0)
    is_sm8x = major == 8 and minor >= 0
    is_sm90 = major == 9 and minor == 0

    return is_sm8x or is_sm90

In [40]:
supports_flash_attention("cuda:2")

False

In [ ]:
from transformers.generation.utils import GenerationConfig

config = codellama_model.config

generation_config = GenerationConfig.from_model_config(config)
print(config)

In [ ]:
vars(generation_config)

In [45]:
del codellama_model
del codellama_tokenizer

import torch
torch.cuda.empty_cache()

NameError: name 'codellama_model' is not defined

## DeepSeek-Coder

In [41]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "deepseek-ai/deepseek-coder-6.7b-instruct"
device = "cuda:0"

deepseek_tokenizer = AutoTokenizer.from_pretrained(model_name)
deepseek_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.97it/s]


#### Fibonacci Example

In [59]:
prompt = "Create a fibonacci sequence method in Java and write JUnit test cases to verify the code runs and computes correctly."#code_test_df["prompted_code"].iloc[7]

inputs = deepseek_tokenizer(prompt,
                             padding=True,
                             truncation=True,
                             return_tensors="pt")

inputs = {key: value.to(device) for key, value in inputs.items()}

In [60]:
outputs = deepseek_model.generate(
    inputs['input_ids'], 
    max_length=1000,
    num_return_sequences=1,
    temperature=0.7,
    eos_token_id=deepseek_tokenizer.eos_token_id
)

generated_text = deepseek_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


Create a fibonacci sequence method in Java and write JUnit test cases to verify the code runs and computes correctly.


```java
public class Fibonacci {
    public static int fibonacci(int n) {
        if (n <= 1) return n;
        else return fibonacci(n-1) + fibonacci(n-2);
    }
}
```

```java
import org.junit.Test;
import static org.junit.Assert.*;

public class FibonacciTest {
    @Test
    public void testFibonacci() {
        assertEquals(0, Fibonacci.fibonacci(0));
        assertEquals(1, Fibonacci.fibonacci(1));
        assertEquals(1, Fibonacci.fibonacci(2));
        assertEquals(2, Fibonacci.fibonacci(3));
        assertEquals(3, Fibonacci.fibonacci(4));
        assertEquals(5, Fibonacci.fibonacci(5));
        assertEquals(8, Fibonacci.fibonacci(6));
        assertEquals(13, Fibonacci.fibonacci(7));
    }
}
```

This code will test the fibonacci method with the first few numbers in the fibonacci sequence. The fibonacci method is a recursive method that computes the nth numbe

#### Example from Dataset

In [42]:
prompt = code_test_df["prompted_code"].iloc[11]

inputs = deepseek_tokenizer(prompt,
                             padding=True,
                             truncation=True,
                             return_tensors="pt")

inputs = {key: value.to(device) for key, value in inputs.items()}

In [43]:
outputs = deepseek_model.generate(
    inputs['input_ids'], 
    max_length=2000,
    num_return_sequences=1,
    temperature=0.7,
    eos_token_id=deepseek_tokenizer.eos_token_id
)

generated_text = deepseek_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text[len(prompt):].strip())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Here is a test class in Java:

```java
import org.junit.Test;
import static org.junit.Assert.*;

public class IdentifierTest {

    @Test
    public void testValidIdentifier() {
        try {
            checkIdentifierLetterAndDigitRequirements("_validIdentifier123");
        } catch (IllegalArgumentException e) {
            fail("Expected no IllegalArgumentException to be thrown");
        }
    }

    @Test(expected = IllegalArgumentException.class)
    public void testInvalidIdentifierStartsWithDigit() {
        checkIdentifierLetterAndDigitRequirements("123invalidIdentifier");
    }

    @Test(expected = IllegalArgumentException.class)
    public void testInvalidIdentifierContainsSpecialChar() {
        checkIdentifierLetterAndDigitRequirements("invalid$Identifier");
    }

    @Test(expected = IllegalArgumentException.class)
    public void testInvalidIdentifierEmpty() {
        checkIdentifierLetterAndDigitRequirements("");
    }
}
```

This test class tests the `checkIdentifie

In [44]:
del deepseek_model
del deepseek_tokenizer

import torch
torch.cuda.empty_cache()